# VQE for Unitary Coupled Cluster using tket

In this tutorial, we will focus on:<br>
- building parameterised ansätze for variational algorithms;<br>
- compilation tools for UCC-style ansätze.

This example assumes the reader is familiar with the Variational Quantum Eigensolver and its application to electronic structure problems through the Unitary Coupled Cluster approach.<br>
<br>
To run this example, you will need `pytket` and `pytket-qiskit`, as well as `scipy` and `sympy`.<br>
<br>
We will start with a basic implementation and then gradually modify it to make it faster, more general, and less noisy. The final solution is given in full at the bottom of the notebook.<br>
<br>
Suppose we have some electronic configuration problem, expressed via a physical Hamiltonian. (The Hamiltonian and excitations in this example were obtained using `qiskit-aqua` version 0.5.2 and `pyscf` for H2, bond length 0.75A, sto3g basis, Jordan-Wigner encoding, with no qubit reduction or orbital freezing.). We express it succinctly using the `pytket` `QubitPauliOperator` class:

In [ ]:
from pytket.utils.operators import QubitPauliOperator
from pytket.circuit import Qubit
from pytket.pauli import Pauli, QubitPauliString

## Qubit H2 Hamiltonian
- This is the Qubit Hamiltonian mapped from the fermionic Hamiltonian under the JW transform 

$$
a^\dagger_j = \bigotimes^{j-1}_i l_i^z \bigotimes \frac{1}{2}(\sigma^\mathbf{x}_j - i\sigma^y_j)
$$

$$
a_j = \bigotimes^{j-1}_i \sigma_i^z \bigotimes \frac{1}{2}(\sigma^\mathbf{x}_j + i\sigma^y_j)
$$


In [ ]:
hamiltonian_op = QubitPauliOperator(
    {
        QubitPauliString(): -0.8153001706270075,
        QubitPauliString({Qubit(0): Pauli.Z}): 0.16988452027940318,
        QubitPauliString({Qubit(1): Pauli.Z}): -0.21886306781219608,
        QubitPauliString({Qubit(2): Pauli.Z}): 0.16988452027940323,
        QubitPauliString({Qubit(3): Pauli.Z}): -0.2188630678121961,
        QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): 0.12005143072546047,
        QubitPauliString({Qubit(0): Pauli.Z, Qubit(2): Pauli.Z}): 0.16821198673715723,
        QubitPauliString({Qubit(0): Pauli.Z, Qubit(3): Pauli.Z}): 0.16549431486978672,
        QubitPauliString({Qubit(1): Pauli.Z, Qubit(2): Pauli.Z}): 0.16549431486978672,
        QubitPauliString({Qubit(1): Pauli.Z, Qubit(3): Pauli.Z}): 0.1739537877649417,
        QubitPauliString({Qubit(2): Pauli.Z, Qubit(3): Pauli.Z}): 0.12005143072546047,
        QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X, Qubit(2): Pauli.X, Qubit(3): Pauli.X}): 0.04544288414432624,
        QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X, Qubit(2): Pauli.Y, Qubit(3): Pauli.Y}): 0.04544288414432624,
        QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y, Qubit(2): Pauli.X, Qubit(3): Pauli.X}): 0.04544288414432624,
        QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y, Qubit(2): Pauli.Y, Qubit(3): Pauli.Y}): 0.04544288414432624
    }
)
nuclear_repulsion_energy = 0.70556961456

Pytket `QubitPauliOperator`

In [ ]:
hamiltonian_op

We would like to define our ansatz for arbitrary parameter values.

The UCC ansatz is derived from the electronic configuration. It sacrifices efficiency of the circuit for the guarantee of physical states and the variational parameters all having some meaningful effect, which helps the classical optimisation to converge.<br>
<br>
This starts by defining the terms of our single and double excitations. These would usually be generated using the orbital configurations, so we will just use a hard-coded example here for the purposes of demonstration.

Single excitations after JW
$$
t_{i\alpha}(a^\dagger_i a_a - a^\dagger_a a_i) = \frac{it_{i\alpha}}{2} \bigotimes_{k=i+1}^{\alpha-1} \sigma^z_k(\sigma^y_i\sigma^\mathbf{x}_\alpha - \sigma^\mathbf{x}_i\sigma^y_\alpha)
$$

Double excitations after JW Mapping

$$
\begin{split}
t_{ij\alpha\beta}(a^\dagger_\alpha a^\dagger_\beta a_i a_j - a^\dagger_j a^\dagger_i a_\alpha a_\beta) =& \frac{it_{i\alpha}}{8}  \bigotimes_{k=i+1}^{j-1} \sigma^z_k  \bigotimes_{l=\alpha+1}^{\beta-1} \sigma^z_l \\
&(\sigma^\mathbf{x}_i \sigma^\mathbf{x}_j \sigma^y_\alpha \sigma^\mathbf{x}_\beta + \sigma^y_i \sigma^\mathbf{x}_j \sigma^y_\alpha \sigma^y_\beta \\
&+\sigma^\mathbf{x}_i \sigma^y_j \sigma^y_\alpha \sigma^y_\beta + \sigma^\mathbf{x}_i \sigma^\mathbf{x}_j \sigma^\mathbf{x}_\alpha \sigma^y_\beta \\
&- \sigma^y_i \sigma^\mathbf{x}_j \sigma^\mathbf{x}_\alpha \sigma^\mathbf{x}_\beta - \sigma^\mathbf{x}_i \sigma^y_j \sigma^\mathbf{x}_\alpha \sigma^\mathbf{x}_\beta\\
&- \sigma^y_i \sigma^y_j \sigma^y_\alpha \sigma^\mathbf{x}_\beta - \sigma^y_i \sigma^y_j \sigma^\mathbf{x}_\alpha \sigma^y_\beta )\\
\end{split}
$$

In [ ]:
q = [Qubit(i) for i in range(4)]
xyii = QubitPauliString([q[0], q[1]], [Pauli.X, Pauli.Y])
yxii = QubitPauliString([q[0], q[1]], [Pauli.Y, Pauli.X])
iixy = QubitPauliString([q[2], q[3]], [Pauli.X, Pauli.Y])
iiyx = QubitPauliString([q[2], q[3]], [Pauli.Y, Pauli.X])
xxxy = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.X, Pauli.Y])
xxyx = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.Y, Pauli.X])
xyxx = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.X, Pauli.X])
yxxx = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.X, Pauli.X])
yyyx = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.Y, Pauli.X])
yyxy = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.X, Pauli.Y])
yxyy = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.Y, Pauli.Y])
xyyy = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.Y, Pauli.Y])

In [ ]:
singles_a = {xyii: 1.0, yxii: -1.0}
singles_b = {iixy: 1.0, iiyx: -1.0}
doubles = {
    xxxy: 0.25,
    xxyx: -0.25,
    xyxx: 0.25,
    yxxx: -0.25,
    yyyx: -0.25,
    yyxy: 0.25,
    yxyy: -0.25,
    xyyy: 0.25,
}

Building the ansatz circuit itself is often done naively by defining the map from each term down to basic gates and then applying it to each term.

Task: Plot this circuit with pytket. What is it doing?
- Identify the basis rotations
- Identify the ZZ..Z operations
- Prove to yourself that these are indeed a parity check and can be used in a pauli gadget

In [ ]:
from pytket import Circuit
def add_operator_term(circuit: Circuit, term: QubitPauliString, angle: float):
    qubits = []
    for q, p in term.map.items():
        if p != Pauli.I:
            qubits.append(q)
            if p == Pauli.X:
                circuit.H(q)
            elif p == Pauli.Y:
                circuit.V(q)
    for i in range(len(qubits) - 1):
        circuit.CX(i, i + 1)
    circuit.Rz(angle, len(qubits) - 1)
    for i in reversed(range(len(qubits) - 1)):
        circuit.CX(i, i + 1)
    for q, p in term.map.items():
        if p == Pauli.X:
            circuit.H(q)
        elif p == Pauli.Y:
            circuit.Vdg(q)

Task: Rewrite this circuit as a custom CircBox

Unitary Coupled Cluster Singles & Doubles ansatz:

- What happens if we had higher excitations?
- is this a good strategy for quantum computing?
- Would you use UCC over another ansatz such as the Hardward efficent ansatz

In [ ]:
def ucc(params):
    ansatz = Circuit(4)
    # Set initial reference state
    ansatz.X(1).X(3)
    # Evolve by excitations
    for term, coeff in singles_a.items():
        add_operator_term(ansatz, term, coeff * params[0])
    for term, coeff in singles_b.items():
        add_operator_term(ansatz, term, coeff * params[1])
    for term, coeff in doubles.items():
        add_operator_term(ansatz, term, coeff * params[2])
    return ansatz

This is already quite verbose, but `pytket` has a neat shorthand construction for these operator terms using the `PauliExpBox` construction. We can then decompose these into basic gates using the `DecomposeBoxes` compiler pass.

In [ ]:
from pytket.circuit import PauliExpBox
from pytket.passes import DecomposeBoxes

In [ ]:
def add_excitation(circ, term_dict, param):
    for term, coeff in term_dict.items():
        qubits, paulis = zip(*term.map.items())
        pbox = PauliExpBox(paulis, coeff * param)
        circ.add_pauliexpbox(pbox, qubits)

UCC ansatz with syntactic shortcuts:

In [ ]:
def ucc(params):
    ansatz = Circuit(4)
    ansatz.X(1).X(3)
    add_excitation(ansatz, singles_a, params[0])
    add_excitation(ansatz, singles_b, params[1])
    add_excitation(ansatz, doubles, params[2])
    DecomposeBoxes().apply(ansatz)
    return ansatz

Let's visualise out ansatz circuit to see the famous "Pauli gadgets".

In [ ]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(ucc([-3.79002933e-05, 2.42964799e-05, 4.63447157e-01]))

The objective function can now be written using a utility method for constructing the measurement circuits and processing for expectation value calculations. For that, we need the Hamiltonian generated above.

Simplified objective function using utilities:

# VQE calculation

- Look at the get operator expectation function in pytket and identify where it s doing the parity checks

In [ ]:
from pytket.utils import get_operator_expectation_value

In [ ]:
def objective(params):
    circ = ucc(params)
    return (
        get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000)
        + nuclear_repulsion_energy
    )

- Optimal values

In [ ]:
arg_values = [-3.79002933e-05, 2.42964799e-05, 4.63447157e-01]

In [ ]:
from pytket.extensions.qiskit import AerBackend
backend = AerBackend()

In [ ]:
energy = objective(arg_values)
print(energy)

This is now the simplest form that this operation can take, but it isn't necessarily the most effective. When we decompose the ansatz circuit into basic gates, it is still very expensive. We can employ some of the circuit simplification passes available in `pytket` to reduce its size and improve fidelity in practice.<br>
<br>
A good example is to decompose each `PauliExpBox` into basic gates and then apply `FullPeepholeOptimise`, which defines a compilation strategy utilising all of the simplifications in `pytket` that act locally on small regions of a circuit. We can examine the effectiveness by looking at the number of two-qubit gates before and after simplification, which tends to be a good indicator of fidelity for near-term systems where these gates are often slow and inaccurate.

In [ ]:
from pytket import OpType
from pytket.passes import FullPeepholeOptimise

In [ ]:
test_circuit = ucc(arg_values)

In [ ]:
print("CX count before", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth before", test_circuit.depth_by_type(OpType.CX))

In [ ]:
FullPeepholeOptimise().apply(test_circuit)

In [ ]:
print("CX count after FPO", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after FPO", test_circuit.depth_by_type(OpType.CX))

These simplification techniques are very general and are almost always beneficial to apply to a circuit if you want to eliminate local redundancies. But UCC ansätze have extra structure that we can exploit further. They are defined entirely out of exponentiated tensors of Pauli matrices, giving the regular structure described by the `PauliExpBox`es. Under many circumstances, it is more efficient to not synthesise these constructions individually, but simultaneously in groups. The `PauliSimp` pass finds the description of a given circuit as a sequence of `PauliExpBox`es and resynthesises them (by default, in groups of commuting terms). This can cause great change in the overall structure and shape of the circuit, enabling the identification and elimination of non-local redundancy.

In [ ]:
from pytket.passes import PauliSimp

In [ ]:
test_circuit = ucc(arg_values)

In [ ]:
print("CX count before", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth before", test_circuit.depth_by_type(OpType.CX))

In [ ]:
PauliSimp().apply(test_circuit)

In [ ]:
print("CX count after PS", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after PS", test_circuit.depth_by_type(OpType.CX))

In [ ]:
FullPeepholeOptimise().apply(test_circuit)

In [ ]:
print("CX count after PS+FPO", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after PS+FPO", test_circuit.depth_by_type(OpType.CX))

To include this into our routines, we can just add the simplification passes to the objective function. The `get_operator_expectation_value` utility handles compiling to meet the requirements of the backend, so we don't have to worry about that here.

Objective function with circuit simplification:

In [ ]:
def objective(params):
    circ = ucc(params)
    PauliSimp().apply(circ)
    FullPeepholeOptimise().apply(circ)
    return (
        get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000)
        + nuclear_repulsion_energy
    )

These circuit simplification techniques have tried to preserve the exact unitary of the circuit, but there are ways to change the unitary whilst preserving the correctness of the algorithm as a whole.<br>
<br>
For example, the excitation terms are generated by trotterisation of the excitation operator, and the order of the terms does not change the unitary in the limit of many trotter steps, so in this sense we are free to sequence the terms how we like and it is sensible to do this in a way that enables efficient synthesis of the circuit. Prioritising collecting terms into commuting sets is a very beneficial heuristic for this and can be performed using the `gen_term_sequence_circuit` method to group the terms together into collections of `PauliExpBox`es and the `GuidedPauliSimp` pass to utilise these sets for synthesis.

In [ ]:
from pytket.passes import GuidedPauliSimp
from pytket.utils import gen_term_sequence_circuit

In [ ]:
def ucc(params):
    singles_params = {qps: params[0] * coeff for qps, coeff in singles.items()}
    doubles_params = {qps: params[1] * coeff for qps, coeff in doubles.items()}
    excitation_op = QubitPauliOperator({**singles_params, **doubles_params})
    reference_circ = Circuit(4).X(1).X(3)
    ansatz = gen_term_sequence_circuit(excitation_op, reference_circ)
    GuidedPauliSimp().apply(ansatz)
    FullPeepholeOptimise().apply(ansatz)
    return ansatz

Adding these simplification routines doesn't come for free. Compiling and simplifying the circuit to achieve the best results possible can be a difficult task, which can take some time for the classical computer to perform.<br>
<br>
During a VQE run, we will call this objective function many times and run many measurement circuits within each, but the circuits that are run on the quantum computer are almost identical, having the same gate structure but with different gate parameters and measurements. We have already exploited this within the body of the objective function by simplifying the ansatz circuit before we call `get_operator_expectation_value`, so it is only done once per objective calculation rather than once per measurement circuit.<br>
<br>
We can go even further by simplifying it once outside of the objective function, and then instantiating the simplified ansatz with the parameter values needed. For this, we will construct the UCC ansatz circuit using symbolic (parametric) gates.

In [ ]:
from sympy import symbols

Symbolic UCC ansatz generation:

In [ ]:
syms = symbols("p0 p1 p2")
singles_a_syms = {qps: syms[0] * coeff for qps, coeff in singles_a.items()}
singles_b_syms = {qps: syms[1] * coeff for qps, coeff in singles_b.items()}
doubles_syms = {qps: syms[2] * coeff for qps, coeff in doubles.items()}
excitation_op = QubitPauliOperator({**singles_a_syms, **singles_b_syms, **doubles_syms})
ucc_ref = Circuit(4).X(1).X(3)
ucc = gen_term_sequence_circuit(excitation_op, ucc_ref)
GuidedPauliSimp().apply(ucc)
FullPeepholeOptimise().apply(ucc)

Objective function using the symbolic ansatz:

In [ ]:
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return (
        get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000)
        + nuclear_repulsion_energy
    )

We have now got some very good use of `pytket` for simplifying each individual circuit used in our experiment and for minimising the amount of time spent compiling, but there is still more we can do in terms of reducing the amount of work the quantum computer has to do. Currently, each (non-trivial) term in our measurement hamiltonian is measured by a different circuit within each expectation value calculation. Measurement reduction techniques exist for identifying when these observables commute and hence can be simultaneously measured, reducing the number of circuits required for the full expectation value calculation.<br>
<br>
This is built in to the `get_operator_expectation_value` method and can be applied by specifying a way to partition the measuremrnt terms. `PauliPartitionStrat.CommutingSets` can greatly reduce the number of measurement circuits by combining any number of terms that mutually commute. However, this involves potentially adding an arbitrary Clifford circuit to change the basis of the measurements which can be costly on NISQ devices, so `PauliPartitionStrat.NonConflictingSets` trades off some of the reduction in circuit number to guarantee that only single-qubit gates are introduced.

In [ ]:
from pytket.partition import PauliPartitionStrat

Objective function using measurement reduction:

In [ ]:
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return (
        get_operator_expectation_value(
            circ,
            operator,
            backend,
            n_shots=4000,
            partition_strat=PauliPartitionStrat.CommutingSets,
        )
        + nuclear_repulsion_energy
    )

At this point, we have completely transformed how our VQE objective function works, improving its resilience to noise, cutting the number of circuits run, and maintaining fast runtimes. In doing this, we have explored a number of the features `pytket` offers that are beneficial to VQE and the UCC method:<br>
- high-level syntactic constructs for evolution operators;<br>
- utility methods for easy expectation value calculations;<br>
- both generic and domain-specific circuit simplification methods;<br>
- symbolic circuit compilation;<br>
- measurement reduction for expectation value calculations.

Exercises:<br>
- Replace the `get_operator_expectation_value` call with its implementation and use this to pull the analysis for measurement reduction outside of the objective function, so our circuits can be fully determined and compiled once. This means that the `symbol_substitution` method will need to be applied to each measurement circuit instead of just the state preparation circuit.<br>
- Use the `SpamCorrecter` class to add some mitigation of the measurement errors. Start by running the characterisation circuits first, before your main VQE loop, then apply the mitigation to each of the circuits run within the objective function.<br>
- Change the `backend` by passing in a `Qiskit` `NoiseModel` to simulate a noisy device. Compare the accuracy of the objective function both with and without the circuit simplification. Try running a classical optimiser over the objective function and compare the convergence rates with different noise models. If you have access to a QPU, try changing the `backend` to connect to that and compare the results to the simulator.